# Building Effective Agents (with Pydantic AI)

Examples for the agentic workflows discussed in
[Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
by [Erik Schluntz](https://github.com/eschluntz) and [Barry Zhang](https://github.com/ItsBarryZ)
of Anthropic, inspired, ported and adapted from the
[code samples](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)
by the authors using [Pydantic AI](https://ai.pydantic.dev/).

## Basic Workflows

In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']

Using AI model: openai:gpt-4o


### Workflow: Prompt Chaining

The first and simplest workflow, chaining, we feed the output of one LLM
call to the next one, and complete our task step by step.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> All LLM calls are handled by the `Agent` class. One feature we take advantage of
> here is a consistent interface for different LLM providers. The model name is
> provided to the constructor and from there, the calls are the same regardless of
> which provider and model we are using.

In [5]:
async def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    agent = Agent(AI_MODEL)
    result = input
    for i, prompt in enumerate(prompts, 1):
        agent = Agent(AI_MODEL, system_prompt=prompt)
        response = await agent.run(f"nInput:\n{result}")
        result = response.output
        show(result, title=f"Step {i}")
    return result

In [6]:
data_processing_steps = [
    """Extract only the numerical values and their associated minion metrics from the text.
    Format each as 'value: minion metric' on a new line.
    Example format:
    92: banana consumption rate
    45%: "BELLO!" frequency increase""",
    
    """Convert all numerical values to percentages where possible for minion analytics.
    If not a percentage or points, convert to decimal (e.g., 92 bananas -> 92%).
    Keep one number per line.
    Example format:
    92%: banana consumption rate
    45%: "BELLO!" frequency increase""",
    
    """Sort all lines in descending order by numerical value for minion performance ranking.
    Keep the format 'value: minion metric' on each line.
    Example:
    92%: banana consumption rate
    87%: yellow outfit compliance""",
    
    """Format the sorted minion data as a markdown table with columns:
    | Minion Metric | Value |
    |:--|--:|
    | Banana Consumption Rate | 92% |"""
]

report = """
Q3 Minion Performance Summary:
Our minion happiness score rose to 92 points this quarter after the banana supply increase.
"BELLO!" frequency grew by 45% compared to last year's metrics.
Yellow outfit compliance is now at 23% improvement in our primary lab.
Minion distraction incidents decreased to 5% from 8% after removing shiny objects.
New evil scheme participation cost is $43 per minion recruit.
Gadget adoption rate increased to 78% among tech-savvy minions.
Gru satisfaction with minions is at 87 points.
Banana storage efficiency improved to 34% reduction in spoilage.
""".strip()

show(report, title="Input text")
formatted_result = await chain(report, data_processing_steps)
show(formatted_result, title="Result")


Input text
----------

Q3 Minion Performance Summary:
Our minion happiness score rose to 92 points this quarter after the banana supply increase.
"BELLO!" frequency grew by 45% compared to last year's metrics.
Yellow outfit compliance is now at 23% improvement in our primary lab.
Minion distraction incidents decreased to 5% from 8% after removing shiny objects.
New evil scheme participation cost is $43 per minion recruit.
Gadget adoption rate increased to 78% among tech-savvy minions.
Gru satisfaction with minions is at 87 points.
Banana storage efficiency improved to 34% reduction in spoilage.


Step 1
------

92: minion happiness score  
45%: "BELLO!" frequency growth  
23%: yellow outfit compliance improvement  
5%: minion distraction incidents  
$43: evil scheme participation cost  
78%: gadget adoption rate  
87: Gru satisfaction  
34%: banana storage efficiency improvement  


Step 2
------

92%: minion happiness score  
45%: "BELLO!" frequency growth  
23%: yellow outfit compli

### Workflow: Routing

In the routing workflow, we rely on an LLM call to decide which
of several options to take. An AI agent is now handling part of
the logic of the app.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> One of the most powerful features of Pydantic AI is ... Pydantic!
> Returing results using structured outputs and validating them against
> the schema specified is built-in. We can specify a `result_type`
> for every agent and receive the parsed and validated Pydantic model
> instance from the LLM call. This is easily one of the best ways of
> interfacing between code and LLM calls, and we'll be using it for
> most workflows.

In [5]:
class RouteSelection(BaseModel):
    reasoning: str = Field(..., description=(
        'Brief explanation of why this ticket should be routed '
        'to a specific team. Consider key terms, user intent, '
        'and urgency level.'
    ))
    selection: str = Field(..., description='The chosen team name')


async def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""

    # First, determine appropriate route using LLM with chain-of-thought
    show(f"{list(routes.keys())}", title="Available Routes")
    
    routing_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'Analyze the input and select the most appropriate support team '
            f'from these options: {list(routes.keys())}'
        ),
        result_type=RouteSelection,
    )
    route_response = await routing_agent.run(input)
    reasoning = route_response.data.reasoning
    route_key = route_response.data.selection.strip().lower()
    
    show(reasoning, title="Routing Analysis")
    show(f"{route_key}", title="Selected Route")
    
    # Process input with selected specialized prompt
    worker_agent = Agent(AI_MODEL, system_prompt=routes[route_key])
    return (await worker_agent.run(input)).data

In [6]:
support_routes = {
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant
    
    Keep responses professional but friendly.""",
    
    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed
    
    Use clear, numbered steps and technical details.""",
    
    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone.""",
    
    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help
    
    Be educational and encouraging in tone."""
}

# Test with different support tickets
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John""",
    
    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",
    
    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    show(ticket, title=f"Ticket {i}")
    response = await route(ticket, support_routes)
    show(response, title=f"Response {i}")

Processing support tickets...


Ticket 1
--------

Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error. 
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to 
    submit a report by end of day.
    - John


Available Routes
----------------

['billing', 'technical', 'account', 'product']


Routing Analysis
----------------

The user is experiencing issues accessing their account, specifically with login credentials. The frustration of repeated login failures and the urgency of needing access suggests that they are locked out due to some authentication problem, which is typically handled by the account support team. Since the query is tightly associated with account access, routing this ticket to the account team is appropriate.


Selected Route
--------------

account


Response 1
----------

Account Support Response:

Dear John,

I understand the urg

### Workflow: Parallelization

LLM calls are typically long-running, I/O-bound operations. They are also stateless,
so in cases where one call doesn't depend on the other, it makes a lot of sense
to parallelize the calls and continue processing once we receive the responses
from all of them.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> `Agent.run()` is an asynchronous call, and in the code here is asnchronous by default.
> Pydantic AI also provides an `Agent.run_sync()` method for cases where you're writing
> synchronous code.


In [9]:
async def parallel(prompt: str, inputs: List[str]) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    agent = Agent(AI_MODEL, system_prompt=prompt)
    results = await asyncio.gather(*[
        agent.run(input)
        for input in inputs
    ])
    return [result.data for result in results]

In [8]:
stakeholders = [
    """Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns""",
    
    """Employees:
    - Job security worries
    - Need new skills
    - Want clear direction""",
    
    """Investors:
    - Expect growth
    - Want cost control
    - Risk concerns""",
    
    """Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions"""
]

impact_results = await parallel(
    """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.""",
    stakeholders
)

for stakeholder, result in zip(stakeholders, impact_results):
    show(result, stakeholder.split(':')[0])


Customers
---------

**Introduction**

As the market undergoes various changes, it is crucial to understand and address the impacts these changes may have on different stakeholder groups. Customers, particularly those who are price sensitive, demand better technology, and are concerned about environmental issues, will be influenced by shifting market dynamics. This analysis will explore these impacts and provide recommendations to help businesses adapt and meet customer expectations effectively.

---

**1. Price Sensitivity**

**Impact:**

- **Inflation and Raw Material Costs:** Rising inflation and increasing raw material costs can lead to higher product prices, which may affect price-sensitive customers adversely, causing them to cut back on spending or switch to more affordable alternatives.
- **Competitive Pricing Pressure:** Competitors might offer similar products at lower prices or provide discounts which may impact market share.

**Recommended Actions:**

- **Cost Optimization